# Semantic Segmentation in Dataloop's Python SDK


In [51]:
#importing Dataloop's Python Package and Numpy
import dtlpy as dl
import numpy as np


In [ ]:
#Logging in to Dataloop (checks if token expired ~24h expiration time for token)
if dl.token_expired():
   dl.login()
#you can also use the simple login: 
#dl.login()

In [52]:
# Importing the Project and Dataset - change with your own project and dataset
project = dl.projects.get(project_name='CreatureHunt')

dataset = project.datasets.get(dataset_name='Creatures')

#Go search for your Item ID, by openinng the dataset inthe  webUI
dataset.open_in_web()


In [53]:
#get the item - change the ID with your Item ID
item = dl.items.get(item_id='63eb694dbb4d84844887871d')

In [54]:

# Create a builder instance
builder = item.annotations.builder()
# Create polygon annotation with label
# with array of points: [[x1, y1], [x2, y2], ..., [xn, yn]]
builder.add(annotation_definition=dl.Polygon(geo=[[200, 50],
                                                  [80, 120],
                                                  [110, 130]],
                                             label='my-label'))
# (optional) create Polyline annotation with label
#builder.add(annotation_definition=dl.Polyline(geo=[[200, 50],
#                                                   [80, 120],
#                                                   [110, 130]],
#                                              label='my-label'))
# Upload polygon to the item
item.annotations.upload(builder)


AnnotationCollection(item=Item(dataset_url='https://gate.dataloop.ai/api/v1/datasets/63e6283b4a03c631b54725ec', created_at='2023-02-14T10:58:21.000Z', dataset_id='63e6283b4a03c631b54725ec', filename='/098-696200529-scale10.00-k_euler_a-sd-v1-5-fp16.png', name='098-696200529-scale10.00-k_euler_a-sd-v1-5-fp16.png', type='file', id='63eb694dbb4d84844887871d', spec=None, creator='myfuncont@gmail.com', _description=None, annotations_count=1), annotations=[Annotation(id='642fdc3f6655cc4a3f63e417', item_id='63eb694dbb4d84844887871d', creator='myfuncont@gmail.com', created_at='2023-04-07T09:02:55.561Z', type='segment', item_height=1024, item_width=1024, label_suggestions=None, _start_frame=0, _start_time=0)])

In [55]:
#You can check the Annotation was added to your image using this line
item.open_in_web()

In [61]:
# Get all polygons

# Pay attention each polygon must have different object ID 
# Here we create a filter that will look for all Polygon Annotations
filters = dl.Filters(resource=dl.FiltersResource.ANNOTATION,
                    field='type',
                    values=dl.AnnotationType.POLYGON)
#we get a list of annotations of a specific item
annotations = item.annotations.list(filters=filters)

In [59]:
#update and list all annotations
annotations.update()
print(annotations)


AnnotationCollection(item=Item(dataset_url='https://gate.dataloop.ai/api/v1/datasets/63e6283b4a03c631b54725ec', created_at='2023-02-14T10:58:21.000Z', dataset_id='63e6283b4a03c631b54725ec', filename='/098-696200529-scale10.00-k_euler_a-sd-v1-5-fp16.png', name='098-696200529-scale10.00-k_euler_a-sd-v1-5-fp16.png', type='file', id='63eb694dbb4d84844887871d', spec=None, creator='myfuncont@gmail.com', _description=None, annotations_count=1), annotations=[Annotation(id='642fdc3f6655cc4a3f63e417', item_id='63eb694dbb4d84844887871d', creator='myfuncont@gmail.com', created_at='2023-04-07T09:02:55.561Z', type='segment', item_height=None, item_width=None, label_suggestions=None, _start_frame=0, _start_time=0)])


In [84]:
# create a blank copy of the h/w size of your image Item
image = np.zeros((item.height, item.width), dtype=np.uint8)

for annotation in annotations:
    # convert Polygon to Segmentation
    seg = dl.Segmentation.from_polygon(geo=annotation.geo,

                                      label=annotation.label,

                                      shape=(item.height, item.width),

                                      attributes=annotation.attributes)

In [85]:
# Add Segmentation geo to Image with the polygon object_id 

image = np.where(seg.geo == 1, annotation.object_id, image)

# Print the segm which should be of type = Segmantation, if everything went fine
print(seg)

### Congrats, you just create a Semantic Segmentation Annotation in Dataloop!